In [1]:
using Pkg
Pkg.add("ProgressBars")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
using ProgressBars
include("proxgrad.jl")
pyplot()

Plots.PyPlotBackend()

In [3]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv")
dataset = dropmissing(dataset)
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)
n = size(dataset,1)
r = .1
cuttoff = Int(round(n*r))
idxs = rand(1:n,n)

# We need to hold out a test dataset to report final results on, that is not used at all for model selection.
# Within k-fold cross validation, train and val datasets will be formed from this main train dataset. 
test_dataset  = dataset[idxs[1:cuttoff],:]
train_dataset = dataset[idxs[cuttoff+1:n],:]

n_train = size(train_dataset,1)
n_test = size(test_dataset,1)

@assert  n_train + n_test == n
@show n_train, n_test
head(dataset)

(n_train, n_test) = (181253, 20139)


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1
,String,Int64,Int64,Int64,Int64,String
1,Table 1a,2010,1,34614,33195,"Salt Lake City, UT"
2,Table 1a,1998,4,30189,31703,"Colorado Springs, CO"
3,Table 1a,1998,4,30198,30852,"Pittsburgh, PA"
4,Table 1a,2009,3,32211,32575,"Las Vegas, NV"
5,Table 1a,1993,4,30255,30852,"Huntsville, AL"
6,Table 1a,2010,4,33198,32575,"Kansas City, MO"


In [34]:
# Do k-fold cross validation and return the average error_metric on the validation set accross the k folds.
function cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(dataset,1)
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    error = 0
    for i in tqdm(1:k)
        val_idxs = r*(i-1)+1:min(r*i,n)
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        # @show i, val_idxs
        # @show tr_low, tr_high
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs]]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs]]
        
        w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_val = X_val * w
        # @show size(y_val)
        # @show size(ŷ_val)
        error += error_metric(ŷ_val, y_val)
    end
    return error / k
end

function extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(X,1)
    
    tr_idxs = dataset[:when].< 2013.5
    val_idxs = dataset[:when].>= 2013.5
    
    X_tr = X[tr_idxs,:]
    y_tr = y[tr_idxs]
    n_tr = size(X_tr,1)

    X_val = X[val_idxs,:]
    y_val = y[val_idxs]
    n_val = size(X_val,1)
    
    @assert n_tr + n_val == n

    w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
    ŷ_val = X_val * w
    # @show size(y_val)
    # @show size(ŷ_val)
    
    return error_metric(ŷ_val, y_val)
end

function train_and_validate(featurizer, loss, regularizer, stepsize, error_metric, mode; k=10, dataset=train_dataset)
    if (mode=="interpolation")
        return cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    else
        return extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    end
end

# For each model in models, do k-fold cross validation and calculate the average error_metric
# on the val set accross the k-folds.
# Each model in model is a tuple of the form (featurizer, loss, regularizer, stepsize),
# where a featurizer is a funciton that takes in a dataset and returns X,y. 
# Returns errors for each model, and the index of the best model
function test_models(models, error_metric,mode;k=10, dataset=train_dataset)
    errors = []
    for model in models
        error = train_and_validate(model...,error_metric,mode;k=k,dataset=dataset)
        errors = [errors; error]
    end
    i = argmin(errors)
    println("The best model is model ",i)
    return errors,i
end

test_models (generic function with 2 methods)

# Feature Engineering and Selection

In [22]:
cat_labels = [:carrier_lg, :carrier_low, :airportid_1, :airportid_2]
cat_labels_sub = [:carrier_lg, :carrier_low]

#Sets of all categories in a particular column
cats_sets = [unique(dataset[:, label]) for label in cat_labels]
cats_sets_sub = [unique(dataset[:, label]) for label in cat_labels_sub]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(length(column), length(cats))
    for i = 1:length(column)
        for j =1:length(cats)
            if column[i] === cats[j]
                result[i, j] = 1
            end    
        end
    end
    result
end

onehot

In [23]:
labels_real = [
    :when, 
    :citymarketid_1, 
    :citymarketid_2,
    :airportid_1,
    :airportid_2,
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]
labels_real_sub = [
    :when, 
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]

7-element Array{Symbol,1}:
 :when
 :nsmiles
 :passengers
 :large_ms
 :fare_lg
 :lf_ms
 :fare_low

In [24]:
# Baseline featurizer
function feats_0(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end

# Only numeric features (treating airportid and cityid as numeric)
function feats_1(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    X = hcat(real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as numerical)
function feats_2(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    cat_vals = hcat([onehot(dataset[:, cat_labels_sub[i]], cats_sets_sub[i]) for i in 1:size(cat_labels_sub, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as categorical)
function feats_3(dataset)
    real_vals = convert(Matrix, dataset[labels_real_sub])
    cat_vals = hcat([onehot(dataset[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

MSE(L1,L2) = sum((L1.-L2).^2) / size(L1,1)

models = [
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .1),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .1),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .2),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .2),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .2),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .3),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .3),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .4),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .4),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .4),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .5),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .5),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .6),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .6),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .6),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .7),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .8),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .8),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .8),
    
    (feats_3, 1/n_train*QuadLoss(), 0.25*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.5*QuadReg(), .9),
    (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .9),
]

errors,i = test_models(models,MSE,"interpolation";k=15)

100.0%┣██████████████████████████████████████████┫ 15/15 [04:54<00:00, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:40<00:00, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:24<00:00, 0.1 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:20<00:00, 0.1 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:36<00:00, 0.1 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:19<00:00, 0.1 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:47<00:00, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [05:05<00:00, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:45<00:00, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [04:58<00:00, 0.0 it/s]┣██████████████▊                             ┫ 5/15 [01:38<04:04, 0.0 it/s]
100.0%┣██████████████████████████████████████████┫ 15/15 [05:12<00:00, 0.0 it/s]
100.0%┣███████████████████████████

(Any[3298.7706837318037, 3344.9843398703783, 3349.547505428101, 3329.4232726401665, 3316.731939845658, 3303.3380665669542, 2917.3447900504098, 2903.5253012967105, 2915.027215413823, 3332.8733037672127  …  2910.890427014236, 2713.335204303255, 2742.71220470165, 2710.8933066291024, 3317.7050081519506, 3316.4787825887965, 3329.23706083248, 3200.206009948353, 3208.812632015834, 3201.029026980858], 21)

In [26]:
sum(train_dataset[:when].< 2013.5) / size(train_dataset,1)

0.788737289865547

In [35]:
# call external extrapolation on models
errors,i = test_models(models,MSE,"extrapolation")

The best model is model 25


(Any[2780.8342180277473, 2780.841445951628, 2780.848673845701, 2780.8342180277473, 2780.841445951628, 2780.848673845701, 3123.6936048510393, 3123.6983442151018, 3123.7030835652886, 2780.8342180277473  …  3123.7030835652886, 2946.9406577359696, 2946.9466016966185, 2946.9525456360375, 2780.8342180277473, 2780.841445951628, 2780.848673845701, 2633.0134051384034, 2633.021899710596, 2633.030394243823], 25)